## All Imports

In [1]:
import numpy as np
import os
import time
from resnet50 import ResNet50
from keras.preprocessing import image
from keras.layers import GlobalAveragePooling2D, Dense, Dropout,Activation,Flatten

from imagenet_utils import preprocess_input
from keras.layers import Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

/home/rabel/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/rabel/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Priting Shape for Single Image

In [2]:
img_path = 'cat_1.jpg'
img = image.load_img(img_path, target_size=(224, 224))

x = image.img_to_array(img)
print(x.shape)

x = np.expand_dims(x, axis=0)
print(x.shape)

x = preprocess_input(x)
print('Input image shape: ', x.shape)

(224, 224, 3)
(1, 224, 224, 3)
Input image shape:  (1, 224, 224, 3)


### Setting dataset directory Path

In [3]:
PATH = '/home/rabel/Downloads/Gallery/Transafer Learning/resNet50/Transfer-Learning-in-keras/'
data_path = PATH + 'data'
data_dir_list = os.listdir(data_path)
print(data_dir_list)

['horses', 'Humans', 'dogs', 'cats']


### Loading Dataset in a single list

In [4]:
img_data_list = []

for dataset in data_dir_list:
    img_list = os.listdir(data_path + '/' + dataset)
    print ('========Loaded the images of dataset-'+'{}=========\n'.format(dataset))
    for img in img_list:
        img_path = data_path + '/' + dataset + '/' + img
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis = 0)
        x = preprocess_input(x)
        #print('Input image shape:', x.shape)
        img_data_list.append(x)
print('All Images are appended')
print('Total imgages:', len(img_data_list))

========Loaded the images of dataset-horses=========

========Loaded the images of dataset-Humans=========

========Loaded the images of dataset-dogs=========

========Loaded the images of dataset-cats=========

All Images are appended
Total imgages: 808


In [5]:
img_data = np.array(img_data_list)
print(img_data.shape)
img_data = np.rollaxis(img_data, 1, 0)
print(img_data.shape)
img_data = img_data[0]
print(img_data.shape)

(808, 1, 224, 224, 3)
(1, 808, 224, 224, 3)
(808, 224, 224, 3)


###  Define the number of classes

In [6]:
num_classes = 4
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')
labels[0:202] = 0
labels[202:404] = 1
labels[404:606] = 2
labels[606:] = 3
names = ['cats','dogs','horses','humans']

In [7]:
# convert class labels to on-hot encoding
Y = np_utils.to_categorical(labels, num_classes)
print(Y)

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


### Shuffling and Spliting dataset

In [8]:
#Shuffle the dataset
x, y = shuffle(img_data, Y, random_state = 2)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)
print('X_train', len(X_train))
print('X_test', len(X_test))
print('y_train', len(y_train))
print('y_test', len(y_test))

X_train 646
X_test 162
y_train 646
y_test 162


### Custom_resnet_model_1

In [9]:
image_input = Input(shape=(224, 224, 3))

model = ResNet50(input_tensor=image_input, include_top=True,weights='imagenet')
model.summary()
last_layer = model.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
out = Dense(num_classes, activation='softmax', name='output_layer')(x)
custom_resnet_model = Model(inputs=image_input,outputs= out)
custom_resnet_model.summary()

TypeError: _obtain_input_shape() got an unexpected keyword argument 'include_top'